### Exercise 04 : A/B-testing
* create a connection to the database using the library sqlite3

In [1]:
import pandas as pd
import sqlite3

connection = sqlite3.connect("../data/checking-logs.sqlite")

* using only one query for each of the groups, create two dataframes: test_results
and control_results with the columns time and avg_diff and only two rows
  * time should have the values: after and before
  * avg_diff contains the average delta among all the users for the time period
  before each of them made their first visit to the page and afterward
  * only take into account the users that have observations before and after
* we still are not using the lab ’project1’

In [2]:
query = """SELECT time, AVG(delta) AS avg_delta
            FROM 
            (SELECT CAST((julianday(some.first_commit_ts) - julianday(datetime(d.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS delta,
                CASE 
                    WHEN first_commit_ts < first_view_ts THEN "before"
                    ELSE "after"
                END AS time
            FROM
                (SELECT uid, MIN(first_view_ts) AS first_view_ts FROM test GROUP BY uid) AS befor
            JOIN
                (SELECT uid, labname, MIN(first_commit_ts)AS first_commit_ts 
                FROM test 
                WHERE NOT labname = "project1" 
                GROUP BY uid, labname) AS some
            
            ON befor.uid = some.uid
            JOIN 
                deadlines d ON some.labname = d.labs)
            GROUP BY time
            """
test_result = pd.io.sql.read_sql(query, connection)
test_result

,time,avg_delta
0,after,-103.40625
1,before,-60.56250


In [3]:
query = """SELECT time, AVG(delta) AS avg_delta
            FROM 
            (SELECT CAST((julianday(some.first_commit_ts) - julianday(datetime(d.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS delta,
                CASE 
                    WHEN first_commit_ts < first_view_ts THEN "before"
                    ELSE "after"
                END AS time
            FROM
                (SELECT uid, MIN(first_view_ts) AS first_view_ts FROM control GROUP BY uid) AS befor
            JOIN
                (SELECT uid, labname, MIN(first_commit_ts)AS first_commit_ts 
                FROM control 
                WHERE NOT labname = "project1" 
                GROUP BY uid, labname) AS some
            
            ON befor.uid = some.uid
            JOIN 
                deadlines d ON some.labname = d.labs)
            GROUP BY time
            """
control_results = pd.io.sql.read_sql(query, connection)
control_results

,time,avg_delta
0,after,-112.710526
1,before,-99.464286
